In [ ]:
import scanpy as sc
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import cupy as cnp
import pandas as pd
import cudf as cpd
import cupyx.scipy.sparse
import matplotlib.pyplot as plt
import scipy.sparse
import gc

In [ ]:
writedir='/data1/zyb/singlecell/'
scfile='/nfs/public/wxz/Consensus_peak/data/process_data/renbin/processed/integrated_renbin.h5ad'
adata=sc.read_h5ad(scfile)

In [ ]:
#get real matrix 
mat=cupyx.scipy.sparse.csc_matrix((586511-560631-1550714+1557129,586511-560631-1550714+1557129),dtype=cnp.float32)
count=0
writedir='/data1/zyb/singlecell/'

mat1=cupyx.scipy.sparse.csr_matrix(cnp.asarray(adata.X[:,560631:586511].minimum(1).todense()).astype('float32'))
mat2=cupyx.scipy.sparse.csr_matrix(cnp.asarray(adata.X[:,1550714:1557129].minimum(1).todense()).astype('float32'))
mat3 = cupyx.scipy.sparse.hstack([mat1, mat2]).tocsr()
del mat1
del mat2
cnp.cuda.Stream.null.synchronize()
#1550714:1557129
for i in range(40):
    mat+=mat3[5000*i:5000*i+5000,:].T.dot(mat3[5000*i:5000*i+5000,:])
    gc.collect()
    cnp.cuda.Stream.null.synchronize()
cnp.cuda.Stream.null.synchronize()
gc.collect()

In [ ]:
cnp.save(writedir+'newmat_dense.npy',mat.todense())
mat=cnp.load(writedir+'newmat_dense.npy')
mat_cpu=mat.get()

In [ ]:
#get random matrix
nonzero_elements_per_col = mat3.getnnz(axis=1)
randmat=cupyx.scipy.sparse.csc_matrix((32295,32295),dtype=cnp.float32)

for i in range(1000):
    t11=cupyx.scipy.sparse.csc_matrix((1000,32295),dtype=cnp.float32)
    gc.collect()
    cnp.cuda.Stream.null.synchronize()
    
    num_samples = 1000
    random_samples = np.random.choice(nonzero_elements_per_col, size=num_samples).astype('int').tolist()
    rows=[]
    cols=[]
    values=[]
    #print(random_samples.shape)
    for j in range(1000):
        random_rows = cnp.ones(random_samples[j])*j
        random_cols = cnp.random.randint(0, 32295, size=random_samples[j])
        random_values = cnp.ones(random_samples[j])
        rows.append(random_rows)
        cols.append(random_cols)
        values.append(random_values)
    t11 += cupyx.scipy.sparse.csc_matrix((cnp.concatenate(values), (cnp.concatenate(rows), cnp.concatenate(cols))), shape=(1000,32295))
    randmat+=t11.T.dot(t11)

In [ ]:
cnp.save(writedir+'random.npy',randmat.todense())
randmat=cnp.load(writedir+'random.npy')
randmat_cpu=randmat.get()

In [ ]:
#Distribution of gene expression times in random
randiag=randmat_cpu.diagonal().get()
plt.hist(randiag, bins=500, color='blue', alpha=0.7)
plt.show()

In [ ]:
#Distribution of gene expression times in real
matdiag=mat_cpu.diagonal().get()
plt.hist(matdiag, bins=500, color='blue', alpha=0.7)
plt.show()

In [ ]:
#filter out genes expressed less than 2000 times
rows_to_keep = cnp.max(mat, axis=1) >= 2000
cols_to_keep = cnp.max(mat, axis=0) >= 2000
mat = mat[rows_to_keep][:, cols_to_keep]

In [ ]:
#condition-express matrix
condmat=mat/mat.diagonal()

In [ ]:
#cluster genes with condmat
from sklearn.cluster import SpectralClustering

graph=cnp.asnumpy(condmat)
graph=(graph+graph.T)/2
n_clusters = 50
spectral = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=42)
labels = spectral.fit_predict(graph)

In [ ]:
unique_labels, label_counts = np.unique(labels, return_counts=True)
for label, count in zip(unique_labels, label_counts):
    print(f"Label {label}: {count} instances")

In [ ]:
# show cond matrix 
data_flattened = cnp.asnumpy(mat).flatten()
data_flattened=data_flattened[data_flattened<1]
plt.hist(data_flattened, bins=500, color='blue', alpha=0.7)
plt.show()

In [ ]:
# show rand cond matrix
randmat=randmat/randmat.diagonal()
randmat_flattened = cnp.asnumpy(randmat).flatten()
randmat_flattened=randmat_flattened[randmat_flattened<1]
plt.hist(randmat_flattened, bins=500, color='blue', alpha=0.7)
plt.show()